**This notebook is an exercise in the [Introduction to Machine Learning](https://www.kaggle.com/learn/intro-to-machine-learning) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/machine-learning-competitions).**

---


# Introduction

Name: Dehan Ammaralda Handiana

This notebook intended for explore the data and try to use other model

In [96]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Data inputted. Now, read it into csv

In [97]:
home_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
home_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

print(home_data.describe())
print(home_data.head())
print(home_data.columns)

In [98]:
missing = home_data.isna()
percent = (missing.sum()/missing.count()*100).sort_values(ascending=False)
missing_columns = percent[percent > 0].index.tolist() # Any
print('Columns which have missing values: \n{0}'.format(missing_columns))

In [99]:
duplicates = home_data.duplicated().sum()
print('Duplicates in train data: {0}'.format(duplicates))

In [100]:
home_data.drop(missing_columns, axis=1, inplace=True)
home_test.drop(missing_columns, axis=1, inplace=True)

In [101]:
s = (home_data.dtypes == 'object')
object_cols = list(s[s].index)
print("Object Columns")
print(object_cols)
print(len(object_cols))

numerical_cols = []
for col in home_data.columns:
    if (col not in object_cols):
        numerical_cols.append(col)
        
print("Numerical Columns")
print(numerical_cols)
print(len(numerical_cols))

In [102]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(8,5)
m = 0
for i, column in enumerate(numerical_cols):
    k, l = divmod(m,5)
    ax = axes[k,l]
    plt.sca(ax)
    plt.scatter(home_data[column],home_data.SalePrice)
    plt.title(column)
    m = m + 1
fig.set_size_inches(24,32)
fig.tight_layout()

In [103]:
# Import helpful libraries
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
# Load the data, and separate the target

y = home_data.SalePrice

# Create X (After completing the exercise, you can return to modify this line!)
features = ['OverallQual', 'GrLivArea', '1stFlrSF', '2ndFlrSF', 'TotalBsmtSF',
            'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF']

# Select columns corresponding to features, and preview the data
X = home_data[features]
X.head()

# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

# Define a random forest model
rf_model = RandomForestRegressor(random_state = 1)
rf_model.fit(train_X, train_y)
rf_val_predictions = rf_model.predict(val_X)
rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)

print("Validation MAE for Random Forest Model: {:,.0f}".format(rf_val_mae))

# Train a model for the competition

The code cell above trains a Random Forest model on **`train_X`** and **`train_y`**.  

Use the code cell below to build a Random Forest model and train it on all of **`X`** and **`y`**.

In [104]:
# To improve accuracy, create a new Random Forest model which you will train on all training data
rf_model_on_full_data = RandomForestRegressor(random_state = 1)


# fit rf_model_on_full_data on all data from the training data
rf_model_on_full_data.fit(X, y)

Now, read the file of "test" data, and apply your model to make predictions.

In [105]:
import numpy as np

y = home_data.SalePrice
features = ['OverallQual', 'GrLivArea', '1stFlrSF', '2ndFlrSF', 'TotalBsmtSF',
            'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF']


# create test_X which comes from test_data but includes only the columns you used for prediction.
# The list of columns is stored in a variable called features
test_X = home_test[features]

test_X.describe()
test_X.isnull().sum()
test_X.fillna(0, inplace=True)
test_X.isnull().sum()
# make predictions which we will submit. 
test_preds = rf_model_on_full_data.predict(test_X)

# Generate a submission

Run the code cell below to generate a CSV file with your predictions that you can use to submit to the competition.

In [106]:
# Run the code to save predictions in the format used for competition scoring

output = pd.DataFrame({'Id': home_test.Id,
                       'SalePrice': test_preds})
output.to_csv('submission.csv', index=False)